In [52]:
%run tools
from tools import *
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import requests
from bs4 import BeautifulSoup
import re
import nltk
from nltk.corpus import stopwords
import glob
import time

In [54]:
# carregando a base de dados
df = load_database()
# criando o modelo
vocabulary, model = bag_of_words(df['content'].values)

In [55]:
X = vocabulary.toarray()
Y = df.label.values

classifier = DecisionTreeClassifier()
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
#Treinar o classificador
classifier.fit(X_train, Y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

In [56]:
#Rodar no conjunto de teste
predict_labels = classifier.predict(X_test)
#Avaliar o classificador
scores = evaluate(Y_test, predict_labels)

In [57]:
scores

[0.8148148148148148,
 0.7727272727272727,
 0.8717948717948718,
 0.8192771084337349]

In [9]:
STOPWORDS = set(stopwords.words("portuguese"))
def clean(text):
    d = re.sub("[^a-zA-Z]", " ", text)
    words = d.lower().split()
    words = [w for w in words if w not in STOPWORDS]
    words = ' '.join(words)
    return words

def content_page(url):
    req = requests.get(url)
    content = req.text
    s = BeautifulSoup(content, 'html.parser')
    data = s.find_all(text=True)
    data = ''.join(data)
    text = data.replace('\n', ' ').replace('\r', '').strip()
    text = ' '.join(text.split())
    return clean(text)

In [10]:
def classify_page(page):
    #Pegar o content de page
    text = content_page(page)
    #Rodar o classificador treinado para esta página
    features = model.transform([text])
    return classifier.predict(features.toarray())[0]

In [13]:
bfs = glob.glob("../Crawler/Bfs/*.csv")
heuristc = glob.glob("../Crawler/Heuristic/*.csv")
crawlers = [bfs, heuristc]

In [24]:
for i, crawler in enumerate(crawlers):
    t = "bfs" if i == 0 else "heuristic"
    for csv in crawler:
        results = []
        df = pd.read_csv(bfs[0]).transpose()
        df = df.reset_index()
        df.columns = ["Links"]

        for index, line in enumerate(df.values):
        print("Classificando " + line[0])
        print((index / df.shape[0]) * 100, "%")
        try:
            results.append([line[0], classify_page(line[0])])
            time.sleep(1)
        except:
            print("Deu pau em: ", line)

        dataframe = pd.DataFrame(results, columns=["url", "label"])
        store = csv.split("/")[3]
        dataframe.to_csv(path_or_buf=f"../Results/re_{t}_{store}", index=False)